In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm


In [9]:
model = SentenceTransformer(
    "all-MiniLM-L6-v2", device="cpu"
)

In [5]:
import pandas as pd
df=pd.read_csv("bigBasketProducts.csv")
df["rating"]=df["rating"].astype(str)
df["rating"].replace({'nan': 'no rating', None: 'no rating'}, inplace=True)
df["description"].fillna("no desc",inplace=True)
df.dropna(inplace=True)

In [10]:
class make_embedding_ds(torch.utils.data.IterableDataset):
    def __init__(self, csv = df):
        """
        will compute embedding using sentence encoder
        
        """
        super(make_embedding_ds).__init__()
        self.csv = csv
        self.total_row = csv.shape[0]
        self.col = csv.columns.to_list()

    def __iter__(self):
        for row_no in range(self.total_row):
            
            #testing: make comment it later
       
            
            
            row = self.csv.iloc[row_no].to_dict()
            product_name = row[self.col[0]]
            story = f"{row[ self.col[0] ]} is of category {row[self.col[1]]} and sub category is {row[self.col[2]]}. {row[self.col[0]]} is type {row[self.col[6]]}. brand of {row[self.col[0]]} is {row[self.col[3]]}, with rating {row[self.col[7]]}. sale price of {row[self.col[0]]} is {row[self.col[4]]} with market price {row[self.col[5]]}, description of {row[self.col[0]]} is {row[self.col[8]]}"
            emb = model.encode(story)
            
            
            yield row_no,product_name,story,emb
    

ds = make_embedding_ds(df)

dataloader = torch.utils.data.DataLoader(ds, num_workers=1,batch_size=128)
json_emb = {
    "payload":[],
    "emb": []
}
batch_no = 1
for row,product_name,story,emb in dataloader:
    
    _batch_len = len(row)
    
    
    
    print(f"batch no-{batch_no} competed")
    batch_no += 1
    
    for index in range(_batch_len):
        
        temp = {}
        temp["id"] = row[index].item()
        temp["product"] = product_name[index]
        temp["story"] = story[index]
        
        json_emb["payload"].append(temp)
        json_emb["emb"].append(emb[index].numpy().tolist())

batch no-1 competed
batch no-2 competed
batch no-3 competed
batch no-4 competed
batch no-5 competed
batch no-6 competed
batch no-7 competed
batch no-8 competed
batch no-9 competed
batch no-10 competed
batch no-11 competed
batch no-12 competed
batch no-13 competed
batch no-14 competed
batch no-15 competed
batch no-16 competed
batch no-17 competed
batch no-18 competed
batch no-19 competed
batch no-20 competed
batch no-21 competed
batch no-22 competed
batch no-23 competed
batch no-24 competed
batch no-25 competed
batch no-26 competed
batch no-27 competed
batch no-28 competed
batch no-29 competed
batch no-30 competed
batch no-31 competed
batch no-32 competed
batch no-33 competed
batch no-34 competed
batch no-35 competed
batch no-36 competed
batch no-37 competed
batch no-38 competed
batch no-39 competed
batch no-40 competed
batch no-41 competed
batch no-42 competed
batch no-43 competed
batch no-44 competed
batch no-45 competed
batch no-46 competed
batch no-47 competed
batch no-48 competed
b

In [11]:
import pickle

# Deserialize the data from the pickle file
with open('space.pkl', 'wb') as file:
    pickle.dump(json_emb, file)

In [12]:
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
import torch
import pickle
import numpy as np

In [13]:
VECTOR_SPACE_PATH = "space.pkl"
with open(VECTOR_SPACE_PATH, 'rb') as file:
    vs = pickle.load(file)
client = QdrantClient(":memory:")
collection_name = "qdrant-space"

collections = client.get_collections()
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
print(client.get_collections())

total_records = len(vs["payload"]) # total records data
_payload = vs["payload"]
_emb = vs["emb"]
ids = list(range(0,total_records))

batch_size = 2  # specify batch size according to your RAM and compute, higher batch size = more RAM usage

client.upsert(
    collection_name=collection_name,
    points=models.Batch(ids=ids, vectors=_emb, payloads=_payload),
)


collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")
# dump qdrant space note keep sentence transformer in environment 

with open('qdrant_space_client.pkl', 'wb') as file:
    pickle.dump(client, file)
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
model = SentenceTransformer(
    "all-MiniLM-L6-v2", device=device
)

with open('encodermodel.pkl', 'wb') as file:
    pickle.dump(model, file)
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

reader = pipeline("question-answering", model=model_name, tokenizer=model_name)
with open('bert-question-answering.pkl', 'wb') as file:
    pickle.dump(reader, file)


collections=[CollectionDescription(name='qdrant-space')]
Vector count in collection: 27553


In [15]:
# VECTOR_SPACE_PATH = "/kaggle/input/chaabi/space.pkl"
BERT_MODEL_PATH = "bert-question-answering.pkl"
ENC_PATH  = "encodermodel.pkl"
QDRANT_PATH  = "qdrant_space_client.pkl"


# with open(VECTOR_SPACE_PATH, 'rb') as file:
#     vs = pickle.load(file)
# bert llm
with open(BERT_MODEL_PATH, 'rb') as file:
    bert = pickle.load(file)
# qdrant clinet to interact qdrant space containing all vectors 

with open(QDRANT_PATH, 'rb') as file:
    qdrant_client = pickle.load(file)

with open(ENC_PATH, 'rb') as file:
    st_encoder = pickle.load(file)
collection_name = "qdrant-space"
def find_close_contexts(question: str, top_k: int) -> list[str]:
    """
    will return contexts contexts close to query 

    Args:
        question (str): What do we want to know?
        top_k (int): top k results will be added 

    Returns:
        context (List[str]):
    """
    try:
        encoded_query = st_encoder.encode(question).tolist() 
        result = qdrant_client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer

        context = [
            [context.payload["product"], context.payload["story"]] for context in result
        ] 
        return context
    except Exception as e:
        print({e})
def tell_me(question: str, context: list[str]):
    """
    Extract the answer from the context for a given question

    Args:
        question (str): _description_
        context (list[str]): _description_
    """
    results = []
    for c in context:
        answer = bert(question=question, context=c[1] )
        answer["product"] = c[0]
        results.append(answer)
        print()

    sorted_result = sorted(results, key=lambda x: x["score"], reverse=True)
    for i in range(len(sorted_result)):
        _out = sorted_result[i]["answer"] 
        _prod = sorted_result[i]["product"]
        _sco = sorted_result[i]["score"]
#         print(f"{i+1}", end=" ")
        print(f"QUERY INPUT: {question}")
        print(f"OUTPUT: {_out} \nPREDICTION SCORE {_sco}\n\nReferred Product: {_prod}\n\n")
        return question,_out,_sco,_prod
queries = []
queries.append("suggest me one product for cleaning vegetables")
queries.append("what is the rating of product Vegetable & Fruit Wash with 100% Natural Action")
queries.append("what is most loved beauty product")
queries.append("price of dove soap")
queries.append("what is most loved beauty product")
queries.append("suggest one Tea Product")
with open("result.txt","w") as result:
    for q in queries:
        
        c = find_close_contexts(q, top_k=1)
        _ques,_out,_sco,_prod = tell_me(q,c)
        result.write(f"QUERY INPUT: {_ques}\nOUTPUT: {_out} \nPREDICTION SCORE {_sco}\n\nReferded Product: {_prod}\n\n")


QUERY INPUT: suggest me one product for cleaning vegetables
OUTPUT: Washing Liquid 
PREDICTION SCORE 0.17999418079853058

Referred Product: 14873



QUERY INPUT: what is the rating of product Vegetable & Fruit Wash with 100% Natural Action
OUTPUT: Disinfectant Spray & Cleaners 
PREDICTION SCORE 0.03999422863125801

Referred Product: 14873



QUERY INPUT: what is most loved beauty product
OUTPUT: Love Beauty & Planet 
PREDICTION SCORE 0.11456327140331268

Referred Product: 11395



QUERY INPUT: price of dove soap
OUTPUT: sale price of 15677 is Dove with market price 115.0 
PREDICTION SCORE 0.31905046105384827

Referred Product: 15677



QUERY INPUT: what is most loved beauty product
OUTPUT: Love Beauty & Planet 
PREDICTION SCORE 0.11456327140331268

Referred Product: 11395



QUERY INPUT: suggest one Tea Product
OUTPUT: 20684 
PREDICTION SCORE 0.16108818352222443

Referred Product: 20684


